# Visualisation partie 2

L'objectif de ce TD est de :
- Créer son premier widget
- Ajouter des éléments d'interactivité à un graphique Plotly
- transfomer le notebook de visualisation interactive en webapp (application web) avec Voilà
- déployer et partager cette application web avec ngrok

# 1) Création d'un premier widget

a) Installation de la librairie Pywi

In [ ]:
import sys
!{sys.executable} -m pip install ipywidgets statsmodels

b) Exécutez le code ci-dessous pour importer la librairie et la renommer widgets. On utilsera

In [ ]:
import ipywidgets as widgets
from IPython.display import display

c) La liste complète des widgets disponible est disponible à [cette adresse](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html).

A l'aide de cette documentation, construisez une variable `my_int_slider` qui contiendra un widget "IntSlider" avec 5 comme valeur par défaut, 1 en minimum et 20 en maximum. On pourra laisser step égal à 1 comme niveau d'incrément.

Utilisez la fonction "display" pour l'afficher.
        

In [ ]:
# IntSlider : curseur pour choisir un entier entre min et max
my_int_slider = widgets.IntSlider(value=5, min=1, max=20, step=1)

display(my_int_slider)

d) A quoi corresponnd le résultat de cette opération ?

In [ ]:
my_int_slider.keys

e) En déduire comment afficher la valeur du slider.

Faites varier la valeur du slider et vérifier qu'en exécutant de nouveau la ligne de code, la valeur a bien changé.

In [ ]:
# .value contient la valeur actuelle du widget (change quand on bouge le curseur)
my_int_slider.value

# 2) Création d'un User Form

Les widgets peuvent servir à créer des user forms pour des utilisateurs finaux.

Cet exercice a pour but de construire un outil qui permet de sélectionner de sélectionner un indice boursier pour afficher un scatter plot avec l'indice d'apple AAPL.

a) Importez les librairies suivantes

In [ ]:
import pandas as pd 
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interactive_output
import io

b) Dans une variable `my_upload_widget`, créez un widget de type Fileupload qui accepte l'extension .csv et un seul fichier à la fois. Puis testez l'import de ce fichier [csv exemple](https://drive.google.com/file/d/1bWMYHBP6pnxB-sztTyRe8gGMszD0GOp4/view?usp=sharing).

In [ ]:
# FileUpload : widget pour importer un fichier depuis l'ordinateur
# accept=".csv" limite aux CSV, multiple=False = un seul fichier à la fois
my_upload_widget = widgets.FileUpload(accept=".csv", multiple=False)

display(my_upload_widget)

c) Le données en sortie sont un peu difficile à extraire.

Exécutez le code ci-dessous pour gagner du temps et posez vos questions au professeur.

In [ ]:
df = pd.read_csv(io.BytesIO(list(my_upload_widget.value)[0]["content"]))
df

d) Représentez dans un graphique Plotly Scatter les donnnées de GOOG versus APPL avec une courbe de tendance (trendlines='ols') pour voir la corrélation entre ces deux indices.

In [ ]:
# Scatter plot : GOOG en x, AAPL en y, avec droite de tendance (corrélation)
fig = px.scatter(df, x="GOOG", y="AAPL", trendline="ols")
fig.show()

e) Créez une fonction `financial_scatter_widget` qui prendra en entrée les paramètres df, x et y, elle affichera le graphique créé précédemment pour les noms de colonnes x et y.

In [ ]:
def financial_scatter_widget(df, x=None, y=None):
    # Même graphique que d) mais paramétré avec x et y
    fig = px.scatter(df, x=x, y=y, trendline="ols")
    fig.show()

financial_scatter_widget(df, x="GOOG", y="AAPL")

f) L'idée est de donner la main à l'utilisateur pour choisir quel indice il souhaite vérifier la correlation avec l'indice APPL.

Créez un widget de type Dropdown, qui permettra de sélectionner le nom d'une colonne du DataFrame créé.

In [ ]:
# Dropdown : menu déroulant avec les noms de colonnes du DataFrame
x_dropdown = widgets.Dropdown(
    options=df.columns.tolist(),
    value='AAPL',
    description='AAPL vs ',
    style= {'description_width': 'initial'},
    disabled=False,
)
display(x_dropdown)

g) A l'aide de la fonction `interact()` de widgets, faites exécutez la fonction `financial_scatter_widget` en fonction de la colonne choisie.

On fixera les paramètres `df`=Le DataFrame récupéré de l'upload  et `y`="APPL" à l'aide de la fonction `fixed()`.

In [ ]:
# interact() connecte le widget au graphique : quand on change le dropdown, le graph se met à jour
# fixed() : paramètres qu'on ne veut pas modifier (df et y restent fixes)
widgets.interact(
    financial_scatter_widget,
    df=fixed(df),
    x=x_dropdown,
    y=fixed("AAPL")
)

# 3) Réalisation du Dashboard restaurateur

Nous allons reprendre les visualisations faites au dernier TP et y ajouter deux éléments d'interaction, l'id du restaurant et les dates de la période sélectionnée.


Done

a) Nous allons reprendre les données du TP précédent. Si besoin, vous pouvez de nouveau les téléchargez à [cette adresse](https://drive.google.com/file/d/1tSA-l-ziWLk90iX48eHne_8aPfx8braI/view?usp=sharing).

b) Exécutez le code suivant de preprocessing des données.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interactive_output
from IPython.display import display

df = pd.read_csv("Tiller_order_data.csv", low_memory=False)

# Transforme les dates en format datetime
df["date_opened"] = pd.to_datetime(df["date_opened"])
df["date_closed"] = pd.to_datetime(df["date_closed"])

# Mettre en index la date d'ouverture de commande
df = df.set_index(df["date_opened"])

In [ ]:
df

c) Exécutez le code ci-dessous qui détermine l'id du restaurant et les dates de la période souhaitée. Ce sont ces paramètres que nous allons rendre sélectionnables grâce aux widgets.

In [ ]:
id_store = 8052
date_debut = "2019-10-01"
date_fin = "2021-12-31"

df_store = df[(df.id_store == id_store) & (df.date_opened >= date_debut) & (df.date_closed < date_fin)]
df_store.shape

d) Changez le code précédent en utilisant deux widgets, un dropdown menu pour choisir le restaurant et un date picker pour choisir la date de début et de fin de la période qu'on souhaite analyser.

Créez la variable `id_store_dropdown` avec un widget de type Dropdown :

In [ ]:
# Dropdown : menu déroulant avec la liste des restaurants
id_store_dropdown = widgets.Dropdown(
    options=df["id_store"].unique().tolist(),
    value=df["id_store"].unique()[0],
    description="Restaurant :",
    style={"description_width": "initial"},
)

In [ ]:
display(id_store_dropdown)

Créez la variable `date_debut_picker` avec un widget de type DatePicker :

In [ ]:
# DatePicker : sélection d'une date de début
date_debut_picker = widgets.DatePicker(
    description="Date début :",
    style={"description_width": "initial"},
)

In [ ]:
display(date_debut_picker)

Créez la variable `date_fin_picker` avec un widget de type DatePicker :

In [ ]:
# DatePicker : sélection d'une date de fin
date_fin_picker = widgets.DatePicker(
    description="Date fin :",
    style={"description_width": "initial"},
)

In [ ]:
display(date_fin_picker)

e) Exécutez le code suivant pour transformer toutes les visualisations réalisées au TP précédent en fonctions

In [ ]:
import plotly.graph_objects as go

def revenue_over_time(df_store):
    df_revenue = df_store[df_store.dim_status=="CLOSED"].resample("D").m_cached_payed.sum()

    fig = px.line(df_revenue,
                  y="m_cached_payed",
                  title="Chiffre d'affaire au cours du temps")
    fig.update_xaxes(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="Last 7days", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.show()

def clients_over_time(df_store):
    df_revenue = df_store[df_store.dim_status=="CLOSED"].resample("D").m_cached_payed.sum()
    df_nb_customer = df_store[df_store.dim_status=="CLOSED"].resample("D")["m_nb_customer"].sum()

    df_average_basket = df_revenue / df_nb_customer

    fig_panier_moyen = go.Scatter(x=df_average_basket.index,
                                  y=df_average_basket,
                                  name="panier moyen",
                                  mode="lines",
                                  line_color="#000000")

    fig = px.bar(df_nb_customer,
                 title="Nombre de clients au cours du temps")
    fig.add_trace(fig_panier_moyen)
    fig.update_xaxes(tickformat="%d/%m/%Y")
    fig.update_xaxes(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.show()

def best_staff(df_store):
    df_store = df_store.copy()
    # Convertir en string d'abord (sinon on ne peut pas mettre "unknown" dans une colonne float)
    df_store["id_waiter"] = df_store["id_waiter"].astype(str).replace("nan", "unknown")

    df_waiter = df_store.groupby("id_waiter", as_index=False)["m_cached_payed"].sum()

    fig = px.pie(
        df_waiter,
        names='id_waiter',
        values='m_cached_payed',
        color='id_waiter',
        title = 'Répartition des commandes non enregistrées par un serveur'
    )
    fig.update_traces(
        textinfo='percent+label'
    )
    fig.show()

f) Faîtes intéragir les widgets précédemment créés, via la fonction 'interact', avec la fonction 'display_dash'.

Cette fonction 'display_dash' filtre le dataframe initial en fonction des valeurs précisées dans les trois widgets, et appelle ensuite les visualisations ci-dessus pour les afficher à partir du dataframe filtré.

Vous devez donc, dans cette fonction 'display_dash' :
- filtrer le dataframe iniitial (df) en fonction des valeurs des widgets id_store_dropdown, date_debut_picker et date_fin_picker
- appeler les fonctions revenue_over_time, clients_over_time et best_staff

In [ ]:
def display_dash(df, id_store_dropdown, date_debut_picker, date_fin_picker):
    # Attendre que les dates soient sélectionnées
    if date_debut_picker is None or date_fin_picker is None:
        print("Veuillez sélectionner une date de début et une date de fin.")
        return
    # Filtrer sur le restaurant et la période choisie via les widgets
    df_filtered = df[(df["id_store"] == id_store_dropdown)]
    df_filtered = df_filtered[df_filtered["date_opened"] >= str(date_debut_picker)]
    df_filtered = df_filtered[df_filtered["date_closed"] < str(date_fin_picker)]
    # Afficher les 3 graphiques
    revenue_over_time(df_filtered)
    clients_over_time(df_filtered)
    best_staff(df_filtered)

In [ ]:
df = pd.read_csv("Tiller_order_data.csv", low_memory=False)

# Transforme les dates en format datetime
df["date_opened"] = pd.to_datetime(df["date_opened"])
df["date_closed"] = pd.to_datetime(df["date_closed"])

# Mettre en index la date d'ouverture de commande
df = df.set_index(df["date_opened"])

In [ ]:
# Cette cellule est complétée en h) plus bas, après la définition de 'ui' en g)
# → passer directement à g) puis h)

g) Organisez votre dashboard en utilisant HBox et VBox de la librairie widgets.

Ils permettent de gérer l'agencement des différentes visualisation :
- placer sur la première ligne le widget : id_store_dropdown
- placer sur la seconde ligne les widgets date_debut_picker et date_fin_picker

In [ ]:
from ipywidgets import HBox, VBox

In [ ]:
# VBox = empile verticalement, HBox = aligne horizontalement
# Ligne 1 : le dropdown du restaurant
# Ligne 2 : les deux date pickers côte à côte
ui = VBox([
    id_store_dropdown,
    HBox([date_debut_picker, date_fin_picker])
])

In [ ]:
ui

h) utiliser maintenant la fonction ['interactive_output'](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#More-control-over-the-user-interface:-interactive_output).
Cette fonction permet d'être plus flexible sur la disposition des widgets.


In [ ]:
# interactive_output connecte les widgets à la fonction display_dash
# Changer un widget relance automatiquement la fonction et met à jour les graphiques
out = interactive_output(
    display_dash,
    {
        "df": fixed(df),
        "id_store_dropdown": id_store_dropdown,
        "date_debut_picker": date_debut_picker,
        "date_fin_picker": date_fin_picker,
    }
)
display(ui, out)

i) Vérifiez que tout fonctionne bien en changeant l'id du restaurant et les dates de la période.
Si tout fonctionne, Bravo vous avez créé votre premier dashboard sous Python !

# 4) Transformation du notebook en application web avec Voilà

Maintenant que nous avons l'ensemble des éléments pour notre dashboard utilisateur, nous allons convertir notre notebook en application web pour qu'on puisse l'exécuter en local et pouvoir interragir avec les éléments.

a) Commencez par installer [Voila](https://github.com/voila-dashboards/voila)

In [ ]:
import sys
!{sys.executable} -m pip install voila

b) Marquez toutes les cellules pour ne garder que les cellules de l'exercice 3, afin de n'afficher que le dashboard de cette partie.

Pour ce faire, vous devrez taggez au préalable les cellules avec un nom de tag. On choisira le tag `hide` dans cet exemple.

Pour cela, cliquez sur "Affichage"-> "Barre d'outil de cellule" -> "Tags"

(Ah... attendez, c'est déjà fait ! On vous fait gagner du temps)

c) Depuis un terminal, lancez la commande :
```
voila --TagRemovePreprocessor.remove_cell_tags hide ce_notebook.ipynb

cd "20260212 - WIDGETS - Extending Notebooks" && voila --TagRemovePreprocessor.remove_cell_tags hide "td_6_visualisation_enoncés_v3.ipynb"

```

# 5) Déploiement de la webapp avec ngrok

On souhaite maintenant pouvoir partager notre webapp sans pour autant la déployer sur serveur sur un autre ordinateur.

Pour cela, nous allons utiliser la librairie ngrok dont la documentation peut être consultée à [cette adresse](https://voila.readthedocs.io/en/stable/deploy.html). Mais nous allons vous guider dans la suite de cette partie.

a) Installer [ngrok](https://ngrok.com/download)

https://ngrok.com/download

b) Unzippez l'archive, puis authentifiez vous avec votre token :

```./ngrok authtoken <token>```

c) Assurez-vous qu'une webapp voilà est déjà lancée en local (cf : commande voila ci-dessus)

d) Lancer un tunnel sur le port 8866 :

```./ngrok http 8866```

e) Cliquez sur le lien affiché dans le terminal (```Forwarding```).
Vous pouvez envoyer cette url à votre voisin, il pourra acceder à votre webapp !

# [Bonus] Déploiement d'une webapp avec Binder

Pour pouvoir utiliser [Binder](https://jupyter.org/binder), il faut au préalable avoir pushé son notebook dans un repo git et avoir rajouté un fichier 'requirements.txt' contenant les librairies nécessaires pour faire tourner votre projet.

Ensuite, il suffit d'aller sur https://mybinder.org/ et :
- dans 'GitHub', copier coller l'URL de votre projet git
- dans 'Path to a notebook file (optional)', écrivez 'voila/render/ce_notebook.ipynb' en précisant bien 'URL' au lieu de 'File'. Celà permet de spécifier à Binder qu'il faut directement utiliser Voila pour créer la webapp.

Cliquez ensuite sur 'lauch'. Une image docker de votre projet sera créer grace au fichier 'requirements.txt'. Vous aurez ensuite un lien pour lancer un conteneur docker autonome qui vous permettra d'acceder à votre application web. Pour chaque utilisateur utilisant ce lien, un conteneur docker sera créé.